In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from typing import Dict, List, Tuple, Optional
from pathlib import Path

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
# pd.options.mode.dtype_backend = 'pyarrow' # use new pyarrow backend instead of numpy
from attrs import define, field, fields, Factory
import tables as tb
from datetime import datetime, timedelta

# Pho's Formatting Preferences
from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
set_pho_preferences_concise()

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange, Epoch
from neuropy.core.ratemap import Ratemap
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.utils.matplotlib_helpers import matplotlib_file_only, matplotlib_configuration, matplotlib_configuration_update
from neuropy.core.neuron_identities import NeuronIdentityTable, neuronTypesList, neuronTypesEnum
from neuropy.utils.mixins.AttrsClassHelpers import AttrsBasedClassHelperMixin, serialized_field, serialized_attribute_field, non_serialized_field, custom_define
from neuropy.utils.mixins.HDF5_representable import HDF_DeserializationMixin, post_deserialize, HDF_SerializationMixin, HDFMixin, HDF_Converter

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core.neurons import NeuronType
from neuropy.core.user_annotations import UserAnnotationsManager
from neuropy.core.position import Position
from neuropy.core.session.dataSession import DataSession
from neuropy.analyses.time_dependent_placefields import PfND_TimeDependent, PlacefieldSnapshot
from neuropy.utils.debug_helpers import debug_print_placefield, debug_print_subsession_neuron_differences, debug_print_ratemap, debug_print_spike_counts, debug_plot_2d_binning, print_aligned_columns
from neuropy.utils.debug_helpers import parameter_sweeps, _plot_parameter_sweep, compare_placefields_info

from pyphocorehelpers.print_helpers import print_object_memory_usage, print_dataframe_memory_usage, print_value_overview_only, DocumentationFilePrinter, print_keys_if_possible, generate_html_string, CapturedException, document_active_variables
from pyphocorehelpers.general_helpers import GeneratedClassDefinitionType, CodeConversion, inspect_callable_arguments

## Pho Programming Helpers:
import inspect
from pyphocorehelpers.general_helpers import inspect_callable_arguments, get_arguments_as_optional_dict, GeneratedClassDefinitionType, CodeConversion
from pyphocorehelpers.print_helpers import DocumentationFilePrinter, TypePrintMode, print_keys_if_possible, debug_dump_object_member_shapes, print_value_overview_only, document_active_variables, CapturedException
from pyphocorehelpers.programming_helpers import IPythonHelpers, PythonDictionaryDefinitionFormat, MemoryManagement


# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_load_session, batch_extended_computations, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme

import pyphoplacecellanalysis.External.pyqtgraph as pg

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons
from pyphoplacecellanalysis.General.Batch.runBatch import BatchSessionCompletionHandler # for `post_compute_validate(...)`
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BasePositionDecoder
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends # for `_perform_long_short_instantaneous_spike_rate_groups_analysis`
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import SingleBarResult, InstantaneousSpikeRateGroupsComputation, TruncationCheckingResults # for `BatchSessionCompletionHandler`, `AcrossSessionsAggregator`
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalPlacefieldGlobalComputationFunctions
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderGlobalComputationFunctions


# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()
# import pylustrator # call `pylustrator.start()` before creating your first figure in code.
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap_pyqtgraph # used in `plot_kourosh_activity_style_figure`
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot, plot_raster_plot
from pyphoplacecellanalysis.General.Mixins.DataSeriesColorHelpers import DataSeriesColorHelpers
from pyphoplacecellanalysis.General.Mixins.DataSeriesColorHelpers import UnitColoringMode, DataSeriesColorHelpers
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import _build_default_tick, build_scatter_plot_kwargs
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.Render2DScrollWindowPlot import Render2DScrollWindowPlotMixin, ScatterItemData
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_extended_programmatic_figures, batch_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends

from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PAPER_FIGURE_figure_1_add_replay_epoch_rasters, PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_3, main_complete_figure_generations

from pyphoplacecellanalysis.SpecificResults.fourthYearPresentation import *

# Jupyter Widget Interactive
import ipywidgets as widgets
from IPython.display import display, HTML
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager
from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import interactive_pipeline_widget, fullwidth_path_widget, interactive_pipeline_files
from pyphocorehelpers.gui.Jupyter.simple_widgets import render_colors

global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/home/halechr/FastData'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data'), Path(r'/home/halechr/cloud/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

Automatic pdb calling has been turned OFF
build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL\TESTING\Logging\debug_com.PhoHale.Spike3D.pipeline.log


# Load Pipeline

In [2]:
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'
local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=a_sess.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

# [*] - indicates bad or session with a problem
# 0, 1, 2, 3, 4, 5, 6, 7, [8], [9], 10, 11, [12], 13, 14, [15], [16], 17, 
# curr_context: IdentifyingContext = good_contexts_list[1] # select the session from all of the good sessions here.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15') # DONE. Very good. Many good Pfs, many good replays.
curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43') # DONE, might be the BEST SESSION, good example session with lots of place cells, clean replays, and clear bar graphs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31') # DONE, Good Pfs but no good replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6') # BAD, 2023-07-14, unsure why still.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19') # DONE, GREAT, both good Pfs and replays!
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25') # DONE, Added replay selections. Very "jumpy" between the starts and ends of the track.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40') # DONE, Added replay selections. A TON of putative replays in general, most bad, but some good.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46') # DONE, added replay selections. Very few (like 12) replays each.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3') # DONE, Good Pfs, no good epochs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30') # DONE, okay replays (selected)
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50') # DONE, very few replays (selected)
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40') # BAD
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54') # DONE, one replay each (selected)
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3') # BAD, Good Pfs strangely despite horrible map, no good epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59') # BAD, No Epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44') # DONE, good. Many good pfs, many good replays. Noticed very strange jumping off the track in the 3D behavior/spikes viewer. Is there something wrong with this session?
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0') # DONE, good?, replays selected, few
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25') # DONE, very few replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54') # DONE, replays selected, quite a few replays but few are very good.


# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25')

local_session_parent_path: Path = local_session_root_parent_path.joinpath(curr_context.animal, curr_context.exper_name) # 'gor01', 'one' - probably not needed anymore
basedir: Path = local_session_parent_path.joinpath(curr_context.session_name).resolve()
print(f'basedir: {str(basedir)}')

# Read if possible:
saving_mode = PipelineSavingScheme.SKIP_SAVING
force_reload = False

# # Force write:
# saving_mode = PipelineSavingScheme.TEMP_THEN_OVERWRITE
# saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
# force_reload = True

## TODO: if loading is not possible, we need to change the `saving_mode` so that the new results are properly saved.

# ==================================================================================================================== #
# Load Pipeline                                                                                                        #
# ==================================================================================================================== #
# with VizTracer(output_file=f"viztracer_{get_now_time_str()}-full_session_LOO_decoding_analysis.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
# epoch_name_includelist = ['maze']
epoch_name_includelist = None
active_computation_functions_name_includelist=['pf_computation',
                                            #    'pfdt_computation',
                                                'firing_rate_trends',
                                                # 'pf_dt_sequential_surprise', 
                                            #    'ratemap_peaks_prominence2d',
                                                'position_decoding', 
                                                # 'position_decoding_two_step', 
                                            #    'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', 'long_short_rate_remapping',
                                            #     'long_short_inst_spike_rate_groups',
                                            #     'long_short_endcap_analysis',
                                            # 'split_to_directional_laps',
]

curr_active_pipeline: NeuropyPipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_includelist=epoch_name_includelist,
                                        computation_functions_name_includelist=active_computation_functions_name_includelist,
                                        saving_mode=saving_mode, force_reload=force_reload,
                                        skip_extended_batch_computations=True, debug_print=False, fail_on_exception=True) # , active_pickle_filename = 'loadedSessPickle_withParameters.pkl'



## Post Compute Validate 2023-05-16:
was_updated = BatchSessionCompletionHandler.post_compute_validate(curr_active_pipeline) ## TODO: need to potentially re-save if was_updated. This will fail because constained versions not ran yet.
if was_updated:
    print(f'was_updated: {was_updated}')
    try:
        curr_active_pipeline.save_pipeline(saving_mode=saving_mode)
    except Exception as e:
        ## TODO: catch/log saving error and indicate that it isn't saved.
        exception_info = sys.exc_info()
        e = CapturedException(e, exception_info)
        print(f'ERROR RE-SAVING PIPELINE after update. error: {e}')


basedir: W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43
Loading loaded session pickle file results : W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\loadedSessPickle.pkl... 

INFO:com.PhoHale.Spike3D.pipeline:NeuropyPipeline.__setstate__(state="{'pipeline_name': 'kdiba_pipeline', 'session_data_type': 'kdiba', '_stage': <pyphoplacecellanalysis.General.Pipeline.Stages.Display.DisplayPipelineStage object at 0x000001FDCDED7AC0>}")
INFO:com.PhoHale.Spike3D.pipeline:select_filters(...) with: []
INFO:com.PhoHale.Spike3D.pipeline:Performing perform_action_for_all_contexts with action EvaluationActions.EVALUATE_COMPUTATIONS on filtered_session with filter named "maze1_odd"...
INFO:com.PhoHale.Spike3D.pipeline:WARNING: skipping computation because overwrite_extant_results=False and active_computation_results[maze1_odd] already exists and is non-None
INFO:com.PhoHale.Spike3D.pipeline:	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
INFO:com.PhoHale.Spike3D.pipeline:Performing perform_action_for_all_contexts with action EvaluationActions.EVALUATE_COMPUTATIONS on fil

done.
Loading pickled pipeline success: W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\loadedSessPickle.pkl.
properties already present in pickled version. No need to save.
pipeline load success!
using provided computation_functions_name_includelist: ['pf_computation', 'firing_rate_trends', 'position_decoding']
curr_active_computation_params.pf_params.computation_epochs: 22 epochs
array([[6.35677, 8.92686],
       [65.785, 72.223],
       [106.426, 112.965],
       [260.414, 281.269],
       [308.896, 313.935],
       [370.624, 375.996],
       [415.704, 422.177],
       [441.164, 445.835],
       [480.803, 489.746],
       [503.559, 509.798],
       [528.45, 533.022],
       [546.403, 551.775],
       [565.554, 570.76],
       [584.542, 591.581],
       [684.407, 688.613],
       [855.014, 858.417],
       [890.015, 895.488],
       [918.243, 922.282],
       [936.163, 940.765],
       [957.918, 962.622],
       [982.841, 992.619],
       [1014.04, 1018.24]])

	 TODO: this will prevent rec

INFO:com.PhoHale.Spike3D.pipeline:	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
INFO:com.PhoHale.Spike3D.pipeline:Performing perform_action_for_all_contexts with action EvaluationActions.EVALUATE_COMPUTATIONS on filtered_session with filter named "maze_any"...
INFO:com.PhoHale.Spike3D.pipeline:WARNING: skipping computation because overwrite_extant_results=False and active_computation_results[maze_any] already exists and is non-None
INFO:com.PhoHale.Spike3D.pipeline:	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
INFO:com.PhoHale.Spike3D.pipeline:Performing global computations...
INFO:com.PhoHale.Spike3D.pipeline:select_filters(...) with: []
INFO:com.PhoHale.Spike3D.pipeline:Performing perform_action_for_all_contexts with action EvaluationActions.EVALUATE_COMPUTATIONS on filte

	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
curr_active_computation_params.pf_params.computation_epochs: 42 epochs
array([[3.05477, 4.72322],
       [45.196, 50.87],
       [84.3034, 91.0442],
       [244.13, 250.671],
       [281.302, 290.778],
       [357.845, 365.118],
       [399.654, 406.36],
       [428.285, 435.323],
       [462.884, 470.192],
       [491.313, 496.985],
       [514.037, 520.176],
       [536.058, 541.798],
       [555.412, 563.486],
       [574.063, 580.235],
       [644.3, 652.009],
       [841.1, 847.706],
       [872.731, 878.203],
       [905.498, 910.802],
       [922.314, 930.589],
       [944.403, 949.976],
       [964.591, 972.998],
       [995.821, 1001.56],
       [1039.93, 1047.01],
       [1064.03, 1068.2],
       [1087.55, 1091.32],
       [1108.04, 1115.64],
       [1151.44, 1157.08],
       [1182.98, 1186.65],
       [1334.76, 1339.94],
  

In [3]:
### GLOBAL COMPUTATIONS:
if not force_reload: # not just force_reload, needs to recompute whenever the computation fails.
    try:
        curr_active_pipeline.load_pickled_global_computation_results()
    except Exception as e:
        exception_info = sys.exc_info()
        e = CapturedException(e, exception_info)
        print(f'cannot load global results: {e}')
        raise

curr_active_pipeline.reload_default_computation_functions()

extended_computations_include_includelist=['pf_computation', 'pfdt_computation', 'firing_rate_trends',
    'pf_dt_sequential_surprise',
     'ratemap_peaks_prominence2d',
    'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding',
    # 'long_short_rate_remapping',
    # 'long_short_inst_spike_rate_groups',
    'long_short_endcap_analysis',
    'spike_burst_detection',
    'split_to_directional_laps',
    'rank_order_shuffle_analysis'
] # do only specified

force_recompute_global = force_reload
# force_recompute_global = True
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=extended_computations_include_includelist, include_global_functions=True, fail_on_exception=False, progress_print=True, force_recompute=force_recompute_global, debug_print=False)
if (len(newly_computed_values) > 0):
    print(f'newly_computed_values: {newly_computed_values}.')
    if (saving_mode.value != 'skip_saving'):
        print(f'Saving global results...')
        try:
            # curr_active_pipeline.global_computation_results.persist_time = datetime.now()
            # Try to write out the global computation function results:
            curr_active_pipeline.save_global_computation_results()
        except Exception as e:
            exception_info = sys.exc_info()
            e = CapturedException(e, exception_info)
            print(f'\n\n!!WARNING!!: saving the global results threw the exception: {e}')
            print(f'\tthe global results are currently unsaved! proceed with caution and save as soon as you can!\n\n\n')
    else:
        print(f'\n\n!!WARNING!!: changes to global results have been made but they will not be saved since saving_mode.value == "skip_saving"')
        print(f'\tthe global results are currently unsaved! proceed with caution and save as soon as you can!\n\n\n')
else:
    print(f'no changes in global results.')

# except Exception as e:
#     exception_info = sys.exc_info()
#     e = CapturedException(e, exception_info)
#     print(f'second half threw: {e}')


# 4m 5.2s for inst fr computations
# Jupyter Widget Interactive
import ipywidgets as widgets
from IPython.display import display
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager
from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import interactive_pipeline_widget, fullwidth_path_widget, interactive_pipeline_files

_pipeline_jupyter_widget = interactive_pipeline_widget(curr_active_pipeline=curr_active_pipeline)
# display(_pipeline_jupyter_widget)
_pipeline_jupyter_widget

Loading loaded session pickle file results : W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\output\global_computation_results.pkl... done.
included includelist is specified: ['pf_computation', 'pfdt_computation', 'firing_rate_trends', 'pf_dt_sequential_surprise', 'ratemap_peaks_prominence2d', 'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', 'long_short_endcap_analysis', 'spike_burst_detection', 'split_to_directional_laps', 'rank_order_shuffle_analysis'], so only performing these extended computations.
Running batch_extended_computations(...) with global_epoch_name: "maze_any"
pf_computation, maze_any already computed.
pfdt_computation, maze_any already computed.
ratemap_peaks_prominence2d, maze_any already computed.
pf_dt_sequential_surprise, maze_any already computed.
spike_burst_detection, maze_any already computed.
firing_rate_trends, maze_any already computed.
split_to_direct

Figure windows disabled (file only)
Figure windows enabled


newly_computed_values: [('long_short_decoding_analyses', 'maze_any'), ('short_long_pf_overlap_analyses', 'maze_any'), ('long_short_fr_indicies_analyses', 'maze_any'), ('jonathan_firing_rate_analysis', 'maze_any'), ('long_short_post_decoding', 'maze_any'), ('long_short_endcap_analysis', 'maze_any')].


In [ ]:
curr_active_pipeline.save_global_computation_results()

In [ ]:
print(curr_active_pipeline.active_completed_computation_result_names) # ['maze1_odd', 'maze2_odd', 'maze_odd', 'maze1_even', 'maze2_even', 'maze_even', 'maze1_any', 'maze2_any', 'maze_any']

In [ ]:
# curr_active_pipeline.active_incomplete_computation_result_status_dicts
curr_active_pipeline.pipeline_compare_dict

In [ ]:
# Force New:
active_2d_plot, active_3d_plot, spike_raster_window = curr_active_pipeline.plot._display_spike_rasters_pyqtplot_2D().values() # included_neuron_ids=EITHER_subset.track_exclusive_aclus

# End Run

In [4]:
# (long_one_step_decoder_1D, short_one_step_decoder_1D), (long_one_step_decoder_2D, short_one_step_decoder_2D) = compute_short_long_constrained_decoders(curr_active_pipeline, recalculate_anyway=True)
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_epoch_context, short_epoch_context, global_epoch_context = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_epoch_name, short_epoch_name, global_epoch_name)]
long_epoch_obj, short_epoch_obj = [Epoch(curr_active_pipeline.sess.epochs.to_dataframe().epochs.label_slice(an_epoch_name.removesuffix('_any'))) for an_epoch_name in [long_epoch_name, short_epoch_name]] #TODO 2023-11-10 20:41: - [ ] Issue with getting actual Epochs from sess.epochs for directional laps: emerges because long_epoch_name: 'maze1_any' and the actual epoch label in curr_active_pipeline.sess.epochs is 'maze1' without the '_any' part.
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_computation_config, short_computation_config, global_computation_config = [curr_active_pipeline.computation_results[an_epoch_name]['computation_config'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D

assert short_epoch_obj.n_epochs > 0, f'long_epoch_obj: {long_epoch_obj}, short_epoch_obj: {short_epoch_obj}'
assert long_epoch_obj.n_epochs > 0, f'long_epoch_obj: {long_epoch_obj}, short_epoch_obj: {short_epoch_obj}'

DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computation_config
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computation_config
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computation_config


In [5]:
## long_short_decoding_analyses:
curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
long_one_step_decoder_1D, short_one_step_decoder_1D, long_replays, short_replays, global_replays, long_shared_aclus_only_decoder, short_shared_aclus_only_decoder, shared_aclus, long_short_pf_neurons_diff, n_neurons, long_results_obj, short_results_obj, is_global = curr_long_short_decoding_analyses.long_decoder, curr_long_short_decoding_analyses.short_decoder, curr_long_short_decoding_analyses.long_replays, curr_long_short_decoding_analyses.short_replays, curr_long_short_decoding_analyses.global_replays, curr_long_short_decoding_analyses.long_shared_aclus_only_decoder, curr_long_short_decoding_analyses.short_shared_aclus_only_decoder, curr_long_short_decoding_analyses.shared_aclus, curr_long_short_decoding_analyses.long_short_pf_neurons_diff, curr_long_short_decoding_analyses.n_neurons, curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj, curr_long_short_decoding_analyses.is_global 
decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333

## Get global `long_short_fr_indicies_analysis`:
long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
long_laps, long_replays, short_laps, short_replays, global_laps, global_replays = [long_short_fr_indicies_analysis_results[k] for k in ['long_laps', 'long_replays', 'short_laps', 'short_replays', 'global_laps', 'global_replays']]
long_short_fr_indicies_df = long_short_fr_indicies_analysis_results['long_short_fr_indicies_df']

## Get global 'long_short_post_decoding' results:
curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result.Flat_epoch_time_bins_mean, expected_v_observed_result.Flat_decoder_time_bin_centers, expected_v_observed_result.num_neurons, expected_v_observed_result.num_timebins_in_epoch, expected_v_observed_result.num_total_flat_timebins, expected_v_observed_result.is_short_track_epoch, expected_v_observed_result.is_long_track_epoch, expected_v_observed_result.short_short_diff, expected_v_observed_result.long_long_diff

jonathan_firing_rate_analysis_result: JonathanFiringRateAnalysisResult = curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis
(epochs_df_L, epochs_df_S), (filter_epoch_spikes_df_L, filter_epoch_spikes_df_S), (good_example_epoch_indicies_L, good_example_epoch_indicies_S), (short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset), new_all_aclus_sort_indicies, assigning_epochs_obj = PAPER_FIGURE_figure_1_add_replay_epoch_rasters(curr_active_pipeline)
neuron_replay_stats_df, short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset = jonathan_firing_rate_analysis_result.get_cell_track_partitions(frs_index_inclusion_magnitude=0.05)

## Update long_exclusive/short_exclusive properties with `long_short_fr_indicies_df`
# long_exclusive.refine_exclusivity_by_inst_frs_index(long_short_fr_indicies_df, frs_index_inclusion_magnitude=0.5)
# short_exclusive.refine_exclusivity_by_inst_frs_index(long_short_fr_indicies_df, frs_index_inclusion_magnitude=0.5)


WARN: 2023-09-28 16:15: - [ ] fix the combination properties. Would work if we directly used the computed _is_L_only and _is_S_only above
WARN: 2023-09-28 16:15: - [ ] fix the combination properties. Would work if we directly used the computed _is_L_only and _is_S_only above


In [6]:
# Unpack all directional variables:
## {"even": "RL", "odd": "LR"}
long_LR_name, short_LR_name, global_LR_name, long_RL_name, short_RL_name, global_RL_name, long_any_name, short_any_name, global_any_name = ['maze1_odd', 'maze2_odd', 'maze_odd', 'maze1_even', 'maze2_even', 'maze_even', 'maze1_any', 'maze2_any', 'maze_any']

# Most popular
# long_LR_name, short_LR_name, long_RL_name, short_RL_name, global_any_name

# Unpacking for `(long_LR_name, long_RL_name, short_LR_name, short_RL_name)`
(long_LR_context, long_RL_context, short_LR_context, short_RL_context) = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
long_LR_epochs_obj, long_RL_epochs_obj, short_LR_epochs_obj, short_RL_epochs_obj, global_any_laps_epochs_obj = [curr_active_pipeline.computation_results[an_epoch_name]['computation_config'].pf_params.computation_epochs for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name, global_any_name)] # note has global also
(long_LR_session, long_RL_session, short_LR_session, short_RL_session) = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)] # sessions are correct at least, seems like just the computation parameters are messed up
(long_LR_results, long_RL_results, short_LR_results, short_RL_results) = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
(long_LR_computation_config, long_RL_computation_config, short_LR_computation_config, short_RL_computation_config) = [curr_active_pipeline.computation_results[an_epoch_name]['computation_config'] for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
(long_LR_pf1D, long_RL_pf1D, short_LR_pf1D, short_RL_pf1D) = (long_LR_results.pf1D, long_RL_results.pf1D, short_LR_results.pf1D, short_RL_results.pf1D)
(long_LR_pf2D, long_RL_pf2D, short_LR_pf2D, short_RL_pf2D) = (long_LR_results.pf2D, long_RL_results.pf2D, short_LR_results.pf2D, short_RL_results.pf2D)
(long_LR_pf1D_Decoder, long_RL_pf1D_Decoder, short_LR_pf1D_Decoder, short_RL_pf1D_Decoder) = (long_LR_results.pf1D_Decoder, long_RL_results.pf1D_Decoder, short_LR_results.pf1D_Decoder, short_RL_results.pf1D_Decoder)


DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computation_config
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computation_config
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computation_config
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computation_config
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computation_config
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WAR

In [7]:
# LR_ripple_rank_order_result, RL_ripple_rank_order_result, LR_laps_rank_order_result, RL_laps_rank_order_result = curr_active_pipeline.global_computation_results.computed_data['RankOrder']

rank_order_results = curr_active_pipeline.global_computation_results.computed_data['RankOrder'] # RankOrderComputationsContainer

# OLD: ODD/EVEN:
# odd_laps_epoch_ranked_aclus_stats_dict, odd_laps_epoch_selected_spikes_fragile_linear_neuron_IDX_dict, odd_laps_long_z_score_values, odd_laps_short_z_score_values, odd_laps_long_short_z_score_diff_values = rank_order_results.odd_laps
# even_laps_epoch_ranked_aclus_stats_dict, even_laps_epoch_selected_spikes_fragile_linear_neuron_IDX_dict, even_laps_long_z_score_values, even_laps_short_z_score_values, even_laps_long_short_z_score_diff_values = rank_order_results.even_laps

# odd_ripple_evts_epoch_ranked_aclus_stats_dict, odd_ripple_evts_epoch_selected_spikes_fragile_linear_neuron_IDX_dict, odd_ripple_evts_long_z_score_values, odd_ripple_evts_short_z_score_values, odd_ripple_evts_long_short_z_score_diff_values = rank_order_results.odd_ripple
# even_ripple_evts_epoch_ranked_aclus_stats_dict, even_ripple_evts_epoch_selected_spikes_fragile_linear_neuron_IDX_dict, even_ripple_evts_long_z_score_values, even_ripple_evts_short_z_score_values, even_ripple_evts_long_short_z_score_diff_values = rank_order_results.even_ripple

# NEW: LR/RL:
LR_laps_epoch_ranked_aclus_stats_dict, LR_laps_epoch_selected_spikes_fragile_linear_neuron_IDX_dict, LR_laps_long_z_score_values, LR_laps_short_z_score_values, LR_laps_long_short_z_score_diff_values = rank_order_results.odd_laps # LR_laps
RL_laps_epoch_ranked_aclus_stats_dict, RL_laps_epoch_selected_spikes_fragile_linear_neuron_IDX_dict, RL_laps_long_z_score_values, RL_laps_short_z_score_values, RL_laps_long_short_z_score_diff_values = rank_order_results.even_laps # RL_laps

LR_ripple_evts_epoch_ranked_aclus_stats_dict, LR_ripple_evts_epoch_selected_spikes_fragile_linear_neuron_IDX_dict, LR_ripple_evts_long_z_score_values, LR_ripple_evts_short_z_score_values, LR_ripple_evts_long_short_z_score_diff_values = rank_order_results.odd_ripple # LR_ripple
RL_ripple_evts_epoch_ranked_aclus_stats_dict, RL_ripple_evts_epoch_selected_spikes_fragile_linear_neuron_IDX_dict, RL_ripple_evts_long_z_score_values, RL_ripple_evts_short_z_score_values, RL_ripple_evts_long_short_z_score_diff_values = rank_order_results.even_ripple # RL_ripple


In [8]:
# Recover from the saved global result:
# Unpacking:
directional_laps_results = curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']
directional_lap_specific_configs, split_directional_laps_dict, split_directional_laps_config_names, computed_base_epoch_names = directional_laps_results.directional_lap_specific_configs, directional_laps_results.split_directional_laps_dict, directional_laps_results.split_directional_laps_config_names, directional_laps_results.computed_base_epoch_names

In [9]:

long_odd_shared_aclus_only_one_step_decoder_1D, long_even_shared_aclus_only_one_step_decoder_1D, short_odd_shared_aclus_only_one_step_decoder_1D, short_even_shared_aclus_only_one_step_decoder_1D = [directional_laps_results.__dict__[k] for k in ['long_odd_shared_aclus_only_one_step_decoder_1D', 'long_even_shared_aclus_only_one_step_decoder_1D', 'short_odd_shared_aclus_only_one_step_decoder_1D', 'short_even_shared_aclus_only_one_step_decoder_1D']]
long_odd_laps_obj, long_even_laps_obj, short_odd_laps_obj, short_even_laps_obj = list(directional_laps_results.split_directional_laps_dict.values())

# print(list(directional_laps_results.__dict__.keys()))
# ['directional_lap_specific_configs', 'split_directional_laps_dict', 'split_directional_laps_contexts_dict', 'split_directional_laps_config_names', 'computed_base_epoch_names', 'long_odd_shared_aclus_only_one_step_decoder_1D', 'long_even_shared_aclus_only_one_step_decoder_1D', 'short_odd_shared_aclus_only_one_step_decoder_1D', 'short_even_shared_aclus_only_one_step_decoder_1D']

# split_directional_laps_config_names
# split_directional_laps_config_names

In [ ]:
curr_active_pipeline.display('_display_1d_placefields', 'maze1_odd')
curr_active_pipeline.display('_display_1d_placefields', 'maze1_even')
curr_active_pipeline.display('_display_1d_placefields', 'maze2_even')
curr_active_pipeline.display('_display_1d_placefields', 'maze2_odd')

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalPlacefieldGlobalDisplayFunctions

In [ ]:
_out = curr_active_pipeline.display(DirectionalPlacefieldGlobalDisplayFunctions._display_directional_laps_overview)


In [ ]:
win = _out['win']
win

In [ ]:
area: pg.DockArea  = win.area


In [ ]:
win.ui.area

In [ ]:
# list(_out.keys())
_out

In [ ]:
# find all exportable items:
_out.plots[

In [ ]:
_out['plots']

In [ ]:
import attr
import attrs
from attrs import define, field, Factory

def create_class_from_dict(class_name, input_dict):
    attributes = {}
    for key, value in input_dict.items():
        attributes[key] = attr.ib(type=type(value), default=value) # , repr=False

    return attrs.make_class(class_name, attributes)

TempGraphicsOutput = create_class_from_dict('TempGraphicsOutput', _out)
TempGraphicsOutput

In [ ]:
instance = TempGraphicsOutput()
print(instance)

grl = instance.plots
grl

In [ ]:
import pyqtgraph.exporters

exporter = pg.exporters.ImageExporter( grl.scene() ) # graphics layout widget (grl)

## 2023-10-19 - Test Instantaneous Spike Rates and their visualizations during the Replay Epochs

In [ ]:
from pyphocorehelpers.indexing_helpers import BinningContainer


replay_instantaneous_time_bin_size_seconds = 0.01
inst_replay_frs: SpikeRateTrends = SpikeRateTrends.init_from_spikes_and_epochs(spikes_df=global_session.spikes_df, filter_epochs=global_replays, included_neuron_ids=EITHER_subset.track_exclusive_aclus.copy(), instantaneous_time_bin_size_seconds=replay_instantaneous_time_bin_size_seconds)
neuron_labels = [str(aclu) for aclu in inst_replay_frs.included_neuron_ids] # labels for plotting the epochs using `BasicBinnedImageRenderingWindow`
# 26 seconds

In [ ]:
epoch_idx = 1
a_replay_inst_frs = inst_replay_frs.inst_fr_df_list[epoch_idx].to_numpy() # (n_epoch_time_bins, n_cells)   ## OLD: #.T # (n_cells, n_epoch_time_bins)
display(a_replay_inst_frs.shape)
a_epoch_timebin_labels = [str(v) for v in np.arange(np.shape(a_replay_inst_frs)[0])]
assert len(a_epoch_timebin_labels) == np.shape(a_replay_inst_frs)[0]
assert len(neuron_labels) == np.shape(a_replay_inst_frs)[1]

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.BinnedImageRenderingWindow import BasicBinnedImageRenderingWindow, LayoutScrollability

out = BasicBinnedImageRenderingWindow(a_replay_inst_frs, None, None, name='a_replay_inst_frs', title="Inst Fr for Replay Epoch", variable_label='Inst FR', scrollability_mode=LayoutScrollability.NON_SCROLLABLE)

In [15]:
active_burst_intervals = curr_active_pipeline.computation_results[global_epoch_name].computed_data['burst_detection']['burst_intervals']
active_burst_intervals

IndexedOrderedDict([(2,     burst_level      t_start        t_end   t_duration                               interval_pair
0             0     0.321515  1731.204408  1730.882892    (0.32151535525918007, 1730.882892348105)
1             1     0.321515     1.733437     1.411921   (0.32151535525918007, 1.4119211971992627)
2           1.0    46.022284    46.395163     0.372879    (46.022284276550636, 0.3728791690664366)
3           1.0    58.234676    62.401811     4.167135      (58.23467610380612, 4.167135146446526)
4           2.0    58.234676    59.038434     0.803758     (58.23467610380612, 0.8037576684728265)
5           3.0    58.234676    58.553949     0.319273     (58.23467610380612, 0.3192727965069935)
..          ...          ...          ...          ...                                         ...
429         4.0   1652.96411   1653.00635      0.04224  (1652.9641098423162, 0.042239978443831205)
430         5.0   1652.96411   1653.00635      0.04224  (1652.9641098423162, 0.042239

In [10]:
# Relative Entropy/Surprise Results:
active_extended_stats = global_results['extended_stats']
active_relative_entropy_results = active_extended_stats['pf_dt_sequential_surprise'] # DynamicParameters
historical_snapshots = active_relative_entropy_results['historical_snapshots']
post_update_times: np.ndarray = active_relative_entropy_results['post_update_times'] # (4152,) = (n_post_update_times,)
snapshot_differences_result_dict = active_relative_entropy_results['snapshot_differences_result_dict']
time_intervals: np.ndarray = active_relative_entropy_results['time_intervals']
surprise_time_bin_duration = (post_update_times[2]-post_update_times[1])
long_short_rel_entr_curves_frames: np.ndarray = active_relative_entropy_results['long_short_rel_entr_curves_frames'] # (4152, 108, 63) = (n_post_update_times, n_neurons, n_xbins)
short_long_rel_entr_curves_frames: np.ndarray = active_relative_entropy_results['short_long_rel_entr_curves_frames'] # (4152, 108, 63) = (n_post_update_times, n_neurons, n_xbins)
flat_relative_entropy_results: np.ndarray = active_relative_entropy_results['flat_relative_entropy_results'] # (149, 63) - (nSnapshots, nXbins)
flat_jensen_shannon_distance_results: np.ndarray = active_relative_entropy_results['flat_jensen_shannon_distance_results'] # (149, 63) - (nSnapshots, nXbins)
flat_jensen_shannon_distance_across_all_positions: np.ndarray = np.sum(np.abs(flat_jensen_shannon_distance_results), axis=1) # sum across all position bins # (4152,) - (nSnapshots)
flat_surprise_across_all_positions: np.ndarray = np.sum(np.abs(flat_relative_entropy_results), axis=1) # sum across all position bins # (4152,) - (nSnapshots)

## Get the placefield dt matrix:
if 'snapshot_occupancy_weighted_tuning_maps' not in active_relative_entropy_results:
	## Compute it if missing:
	occupancy_weighted_tuning_maps_over_time = np.stack([placefield_snapshot.occupancy_weighted_tuning_maps_matrix for placefield_snapshot in historical_snapshots.values()])
	active_relative_entropy_results['snapshot_occupancy_weighted_tuning_maps'] = occupancy_weighted_tuning_maps_over_time
else:
	occupancy_weighted_tuning_maps_over_time = active_relative_entropy_results['snapshot_occupancy_weighted_tuning_maps'] # (n_post_update_times, n_neurons, n_xbins)


In [11]:
# Time-dependent
long_pf1D_dt, short_pf1D_dt, global_pf1D_dt = long_results.pf1D_dt, short_results.pf1D_dt, global_results.pf1D_dt
long_pf2D_dt, short_pf2D_dt, global_pf2D_dt = long_results.pf2D_dt, short_results.pf2D_dt, global_results.pf2D_dt
global_pf1D_dt: PfND_TimeDependent = global_results.pf1D_dt
global_pf2D_dt: PfND_TimeDependent = global_results.pf2D_dt

In [12]:
## long_short_endcap_analysis: checks for cells localized to the endcaps that have their placefields truncated after shortening the track
truncation_checking_result: TruncationCheckingResults = curr_active_pipeline.global_computation_results.computed_data.long_short_endcap
disappearing_endcap_aclus = truncation_checking_result.disappearing_endcap_aclus
# disappearing_endcap_aclus
trivially_remapping_endcap_aclus = truncation_checking_result.minor_remapping_endcap_aclus
# trivially_remapping_endcap_aclus
significant_distant_remapping_endcap_aclus = truncation_checking_result.significant_distant_remapping_endcap_aclus
# significant_distant_remapping_endcap_aclus
appearing_aclus = jonathan_firing_rate_analysis_result.neuron_replay_stats_df[jonathan_firing_rate_analysis_result.neuron_replay_stats_df['track_membership'] == SplitPartitionMembership.RIGHT_ONLY].index
# appearing_aclus

In [14]:
significant_distant_remapping_endcap_aclus

Int64Index([25, 43, 51, 53, 84, 89, 92], dtype='int64')

In [13]:
from pyphoplacecellanalysis.SpecificResults.fourthYearPresentation import fig_remapping_cells

graphics_output_dict = fig_remapping_cells(curr_active_pipeline=curr_active_pipeline)

override_sortby: peak_long is being used.
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEBUG: new_all_aclus_sort_indicies: [1 4 7 0 6 2 3 5]
active_context: kdiba_gor01_one_2006-6-09_1-22-43_disappear_endcap
	 saved C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting\2023-11-14\kdiba\gor01\one\2006-6-09_1-22-43\disappear_endcap_display_short_long_pf1D_comparison.png
override_sortby: peak_long is being used.
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEBUG: new_all_aclus_sort_indicies: [6 0 5 1 4 2 3]
active_context: kdiba_gor01_one_2006-6-09_1-22-43_sig_re

In [ ]:
# 2023-11-10 - Adds "LxC_PBEsDeltaMinus" for PBEs 


temp = add_extra_spike_rate_trends(curr_active_pipeline)
temp


# 🟢 2023-10-20 - Z-Score Comparisons with Neuron_ID Shuffled templates
1. Take the intersection of the long and short templates to get only the common cells
2. Determine the long and short "tempaltes": this is done by ranking the aclus for each by their placefields' center of mass. `compute_placefield_center_of_masses`
	2a. `long_pf_peak_ranks`, `short_pf_peak_ranks` - there are one of each of these for each shared aclu.
3. Generate the unit_id shuffled (`shuffled_aclus`, `shuffle_IDXs`) ahead of time to use to shuffle the two templates during the epochs.
4. For each replay event, take each shuffled template
	4a. Iterate through each shuffle and obtain the shuffled templates like `long_pf_peak_ranks[epoch_specific_shuffled_indicies]`, `short_pf_peak_ranks[epoch_specific_shuffled_indicies]`
	4b. compute the spearman rank-order of the event and each shuffled template, and accumulate the results in `long_spearmanr_rank_stats_results`, `short_spearmanr_rank_stats_results`

5. After we're done with the shuffle loop, accumulate the results and convert to the right output format.

6. When all epochs are done, loop through the results (the epochs again) and compute the z-scores for each epoch so they can be compared to each other. Keep track of the means and std_dev for comparisons later, and subtract the two sets of z-scores (long/short) to get the delta_Z for each template.

7. TODO: Next figure out what to do with the array of z-scores and delta_Z. We have:
	n_epochs sets of results
		n_shuffles scores of delta_Z



## Convo with Kamran 2023-10-23:
- Use directional templates **
- No need to worry about re-ranking
[X] Plot the long and short separately in addition to the difference, so we show significant reqplay on each as a sanity check
[X] Absolute value difference?
[X] Fisher transform the correlation values (check if there is a difference) because correlation coefficients aren't going to be normally distributed.
	[ ] Then Z-score releative to fisher.

- T-test to compare to mean of zero (if looking at the difference)

In [ ]:
## Concerns:
# 1. Permutation recommended over shuffling for small numbers of ids
# 2.


In [19]:
from nptyping import NDArray
from attrs import define, field, Factory, astuple
import scipy.stats
from scipy import ndimage
from neuropy.utils.misc import build_shuffled_ids # used in _SHELL_analyze_leave_one_out_decoding_results
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import pho_stats_paired_t_test
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import compute_shuffled_rankorder_analyses, build_track_templates_for_shuffle, compute_shuffled_rankorder_analyses
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import TrackTemplates, RankOrderAnalyses, ShuffleHelper, Zscorer
from pyphoplacecellanalysis.Pho2D.PyQtPlots.Extensions.pyqtgraph_helpers import build_pyqtgraph_epoch_indicator_regions
from pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig import DisplayColorsEnum
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _helper_add_long_short_session_indicator_regions
from pyphocorehelpers.DataStructure.general_parameter_containers import VisualizationParameters, RenderPlotsData, RenderPlots # PyqtgraphRenderPlots
from pyphocorehelpers.gui.PhoUIContainer import PhoUIContainer
from pyphocorehelpers.DataStructure.RenderPlots.PyqtgraphRenderPlots import PyqtgraphRenderPlots
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.RankOrderDebugger import GenericPyQtGraphContainer
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.RankOrderDebugger import GenericPyQtGraphScatterClicker
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.DockAreaWrapper import DockAreaWrapper, PhoDockAreaContainingWindow
from pyphoplacecellanalysis.GUI.PyQtPlot.DockingWidgets.DynamicDockDisplayAreaContent import CustomDockDisplayConfig
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderComputationsContainer, RankOrderResult

In [20]:
# Recover from the saved global result:
directional_laps_results = curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']
long_odd_shared_aclus_only_one_step_decoder_1D, long_even_shared_aclus_only_one_step_decoder_1D, short_odd_shared_aclus_only_one_step_decoder_1D, short_even_shared_aclus_only_one_step_decoder_1D = [directional_laps_results.__dict__[k] for k in ['long_odd_shared_aclus_only_one_step_decoder_1D', 'long_even_shared_aclus_only_one_step_decoder_1D', 'short_odd_shared_aclus_only_one_step_decoder_1D', 'short_even_shared_aclus_only_one_step_decoder_1D']]
track_templates: TrackTemplates = TrackTemplates.init_from_paired_decoders(LR_decoder_pair=(long_odd_shared_aclus_only_one_step_decoder_1D, short_odd_shared_aclus_only_one_step_decoder_1D), RL_decoder_pair=(long_even_shared_aclus_only_one_step_decoder_1D, short_even_shared_aclus_only_one_step_decoder_1D))

In [21]:
# Unpack all directional variables:
## {"even": "RL", "odd": "LR"}
long_LR_name, short_LR_name, global_LR_name, long_RL_name, short_RL_name, global_RL_name, long_any_name, short_any_name, global_any_name = ['maze1_odd', 'maze2_odd', 'maze_odd', 'maze1_even', 'maze2_even', 'maze_even', 'maze1_any', 'maze2_any', 'maze_any']

# Most popular
# long_LR_name, short_LR_name, long_RL_name, short_RL_name, global_any_name

# Unpacking for `(long_LR_name, long_RL_name, short_LR_name, short_RL_name)`
(long_LR_context, long_RL_context, short_LR_context, short_RL_context) = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
long_LR_epochs_obj, long_RL_epochs_obj, short_LR_epochs_obj, short_RL_epochs_obj, global_any_laps_epochs_obj = [curr_active_pipeline.computation_results[an_epoch_name]['computation_config'].pf_params.computation_epochs for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name, global_any_name)] # note has global also
(long_LR_session, long_RL_session, short_LR_session, short_RL_session) = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)] # sessions are correct at least, seems like just the computation parameters are messed up
(long_LR_results, long_RL_results, short_LR_results, short_RL_results) = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
(long_LR_computation_config, long_RL_computation_config, short_LR_computation_config, short_RL_computation_config) = [curr_active_pipeline.computation_results[an_epoch_name]['computation_config'] for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
(long_LR_pf1D, long_RL_pf1D, short_LR_pf1D, short_RL_pf1D) = (long_LR_results.pf1D, long_RL_results.pf1D, short_LR_results.pf1D, short_RL_results.pf1D)
(long_LR_pf2D, long_RL_pf2D, short_LR_pf2D, short_RL_pf2D) = (long_LR_results.pf2D, long_RL_results.pf2D, short_LR_results.pf2D, short_RL_results.pf2D)
(long_LR_pf1D_Decoder, long_RL_pf1D_Decoder, short_LR_pf1D_Decoder, short_RL_pf1D_Decoder) = (long_LR_results.pf1D_Decoder, long_RL_results.pf1D_Decoder, short_LR_results.pf1D_Decoder, short_RL_results.pf1D_Decoder)


DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computation_config
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computation_config
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computation_config
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computation_config
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computation_config
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WAR

# ❇️🆕 READY/NEXT: 2023-11-10 - All directional pf1D works for merging all four 1D templates!!

In [22]:
from neuropy.analyses.placefields import PfND
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BasePositionDecoder
from neuropy.utils.mixins.time_slicing import TimeColumnAliasesProtocol
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import DecodedFilterEpochsResult

# Use the four epochs to make to a pseudo-y:
all_directional_decoder_names = ['long_LR', 'long_RL', 'short_LR', 'short_RL']
all_directional_pf1D = PfND.build_merged_directional_placefields(deepcopy(long_LR_pf1D), deepcopy(long_RL_pf1D), deepcopy(short_LR_pf1D), deepcopy(short_RL_pf1D), debug_print=False)
all_directional_pf1D_Decoder = BasePositionDecoder(all_directional_pf1D, setup_on_init=True, post_load_on_init=True, debug_print=False)


WARN: TODO 2023-11-10 16:31 CORRECTNESS ISSUE: - [ ] When we filter on the speed_thresh and remove the spikes, we must also remove the time below this threshold from consideration!
using self.config.grid_bin_bounds: ((36.58620390950715, 248.91627658974846), (0, 4))


In [ ]:

## Combine the non-directional PDFs and renormalize to get the directional PDF:
# Inputs: long_LR_pf1D, long_RL_pf1D
all_directional_decoder_names = ['long_LR', 'long_RL']
long_directional_pf1D = PfND.build_merged_directional_placefields(deepcopy(long_LR_pf1D), deepcopy(long_RL_pf1D), debug_print=False)
long_directional_pf1D_Decoder = BasePositionDecoder(long_directional_pf1D, setup_on_init=True, post_load_on_init=True, debug_print=False)
# takes 6.3 seconds

In [24]:
all_directional_pf1D_Decoder.ratemap.dims_coord_tuple 


(56, 4)

In [31]:
all_directional_pf1D_Decoder.ratemap.ndim
all_directional_pf1D_Decoder.ratemap.plot()

([<Figure size 6400x1100 with 341 Axes>,
  <Figure size 6400x1100 with 341 Axes>],
 [{2: {'axs': [<Axes: label='Cell 2-(shk 1, clu 2) - 9.4 Hz'>],
    'image': <matplotlib.image.AxesImage at 0x200ea921c10>,
    'title_obj': <matplotlib.offsetbox.AnchoredText at 0x200ea972df0>},
   3: {'axs': [<Axes: label='Cell 3-(shk 1, clu 3) - 17.35 Hz'>],
    'image': <matplotlib.image.AxesImage at 0x200ea978790>,
    'title_obj': <matplotlib.offsetbox.AnchoredText at 0x200ea978b80>},
   4: {'axs': [<Axes: label='Cell 4-(shk 1, clu 4) - 7.82 Hz'>],
    'image': <matplotlib.image.AxesImage at 0x200ea97d4f0>,
    'title_obj': <matplotlib.offsetbox.AnchoredText at 0x200ea97d880>},
   5: {'axs': [<Axes: label='Cell 5-(shk 1, clu 10) - 15.61 Hz'>],
    'image': <matplotlib.image.AxesImage at 0x200ea984220>,
    'title_obj': <matplotlib.offsetbox.AnchoredText at 0x200ea984610>},
   6: {'axs': [<Axes: label='Cell 6-(shk 1, clu 11) - 6.59 Hz'>],
    'image': <matplotlib.image.AxesImage at 0x200ea984e80>,
 

c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\ipykernel\eventloops.py:128: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  el.exec() if hasattr(el, 'exec') else el.exec_()


In [28]:
all_directional_pf1D_Decoder.pf.occupancy #.shape: (56, 4)

array([[0, 0.834175, 12.3458, 0],
       [0, 3.10313, 8.24165, 0],
       [0, 3.06976, 3.37007, 0],
       [0, 4.20424, 2.9363, 0],
       [0, 5.00505, 5.20525, 0],
       [0, 0.100101, 0.300303, 0],
       [0, 0.033367, 0, 0],
       [0, 0, 0, 0],
       [0, 1.16785, 1.46815, 0],
       [0, 0.567239, 0.767441, 0.400404],
       [0, 2.33569, 2.70273, 0.734074],
       [0, 1.80182, 3.0364, 28.9626],
       [0, 1.26795, 1.66835, 14.2811],
       [0, 1.70172, 2.03539, 8.40849],
       [0, 1.76845, 1.90192, 6.03943],
       [0, 2.26896, 2.80283, 6.6734],
       [0, 1.80182, 1.03438, 4.73812],
       [0, 1.36805, 1.53488, 3.87057],
       [0, 1.60162, 2.10212, 4.30434],
       [0, 1.66835, 1.06774, 3.0364],
       [0, 1.60162, 1.16785, 2.80283],
       [0, 1.40141, 1.50152, 3.06976],
       [0, 1.63498, 1.20121, 2.8362],
       [0, 1.96865, 1.10111, 2.70273],
       [0, 1.20121, 1.06774, 2.33569],
       [0, 2.03539, 1.26795, 3.0364],
       [0, 1.56825, 1.33468, 2.56926],
       [0, 1.4681

array([0.834174, 3.10313, 3.06976, 4.20424, 5.00505, 0.100101, 0.033367, 0, 1.16784, 0.567239, 2.33569, 1.80182, 1.26794, 1.70172, 1.76845, 2.26895, 1.80182, 1.36805, 1.60161, 1.66835, 1.60161, 1.40141, 1.63498, 1.96865, 1.20121, 2.03539, 1.56825, 1.46815, 1.70172, 1.26794, 1.53488, 1.26794, 1.50151, 1.60161, 1.66835, 1.40141, 1.90192, 1.66835, 1.90192, 1.03438, 1.03438, 1.73508, 1.23458, 1.73508, 1.90192, 0.867541, 2.06875, 2.03539, 2.16885, 3.20323, 3.1365, 3.16986, 5.70575, 16.8837, 11.2447, 3.67037])

In [ ]:
long_LR_pf1D.occupancy

In [ ]:
long_LR_pf1D.plot_occupancy()

In [34]:
long_RL_pf1D.plot_occupancy()

(<Figure size 640x480 with 1 Axes>,
 <Axes: title={'center': '(speedThresh_10.00, gridBin_3.81, smooth_2.00, frateThresh_1.00)grid_bin_bounds_((36.58620390950715, 248.91627658974846), (132.81136363636367, 149.2840909090909)), is_directional_False'}>)

##  🖼️ Plot a debug view for directional laps. Shows the laps, the pf1Ds, ...


In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.DockAreaWrapper import DockAreaWrapper, PhoDockAreaContainingWindow
from pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig import DisplayColorsEnum, LongShortDisplayConfigManager
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.GraphicsWidgets.EpochsEditorItem import EpochsEditor # perform_plot_laps_diagnoser
from pyphoplacecellanalysis.External.pyqtgraph.dockarea.Dock import Dock, DockDisplayConfig
from pyphoplacecellanalysis.GUI.PyQtPlot.DockingWidgets.DynamicDockDisplayAreaContent import CustomDockDisplayConfig

epochs_editor = EpochsEditor.init_from_session(global_session, include_velocity=True, include_accel=False)
root_dockAreaWindow, app = DockAreaWrapper.wrap_with_dockAreaWindow(epochs_editor.plots.win, None, title='Pho Directional Laps Templates')

# track_templates.long_LR_decoder.pf

def _get_decoder_sorted_pfs(a_decoder):
	ratemap = a_decoder.pf.ratemap
	CoM_sort_indicies = np.argsort(ratemap.peak_tuning_curve_center_of_masses) # get the indicies to sort the placefields by their center-of-mass (CoM) location
	# CoM_sort_indicies.shape # (n_neurons,)
	return ratemap.pdf_normalized_tuning_curves[CoM_sort_indicies, :]

decoders_dict = {'long_LR': track_templates.long_LR_decoder,
	'long_RL': track_templates.long_RL_decoder,
	'short_LR': track_templates.short_LR_decoder,
	'short_RL': track_templates.short_RL_decoder,
}

## Plot the placefield 1Ds as heatmaps and then wrap them in docks and add them to the window:
_out_pf1D_heatmaps = {}
for a_decoder_name, a_decoder in decoders_dict.items():
	_out_pf1D_heatmaps[a_decoder_name] = visualize_heatmap_pyqtgraph(_get_decoder_sorted_pfs(a_decoder), title=f'{a_decoder_name}_pf1Ds', show_value_labels=False, show_xticks=False, show_yticks=False, show_colorbar=False, win=None, defer_show=True)

even_dock_config = CustomDockDisplayConfig(custom_get_colors_callback_fn=DisplayColorsEnum.Laps.get_even_dock_colors)
odd_dock_config = CustomDockDisplayConfig(custom_get_colors_callback_fn=DisplayColorsEnum.Laps.get_odd_dock_colors)


_out_dock_widgets = {}
dock_configs = (even_dock_config, odd_dock_config, even_dock_config, odd_dock_config)
dock_add_locations = (['left'], ['left'], ['right'], ['right'])

for i, (a_decoder_name, a_heatmap) in enumerate(_out_pf1D_heatmaps.items()):
	_out_dock_widgets[a_decoder_name] = root_dockAreaWindow.add_display_dock(identifier=a_decoder_name, widget=a_heatmap[0], dockSize=(300,200), dockAddLocationOpts=dock_add_locations[i], display_config=dock_configs[i])


# Outputs: root_dockAreaWindow, app, epochs_editor, _out_pf1D_heatmaps, _out_dock_widgets

In [ ]:
# should easily be able to plot the placefields, the occupancy:

all_directional_pf1D_Decoder.ratemap

np.print_options


In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.DockAreaWrapper import DockAreaWrapper, PhoDockAreaContainingWindow
from pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig import DisplayColorsEnum, LongShortDisplayConfigManager
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.GraphicsWidgets.EpochsEditorItem import EpochsEditor # perform_plot_laps_diagnoser
from pyphoplacecellanalysis.External.pyqtgraph.dockarea.Dock import Dock, DockDisplayConfig
from pyphoplacecellanalysis.GUI.PyQtPlot.DockingWidgets.DynamicDockDisplayAreaContent import CustomDockDisplayConfig
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap_pyqtgraph # used in `plot_kourosh_activity_style_figure`
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import TrackTemplates # _display_directional_laps_overview

# uses `global_session`
epochs_editor = EpochsEditor.init_from_session(global_session, include_velocity=False, include_accel=False)

In [ ]:
# Visualization:
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap, visualize_heatmap_pyqtgraph
from pyphoplacecellanalysis.Pho2D.PyQtPlots.plot_placefields import pyqtplot_plot_image_array, display_all_pf_2D_pyqtgraph_binned_image_rendering
from pyphoplacecellanalysis.GUI.PyQtPlot.BinnedImageRenderingWindow import BasicBinnedImageRenderingWindow, LayoutScrollability


In [ ]:

def _display_fn_BasePositionDecoder_visualize_heatmap_pyqtgraph(a_decoder: BasePositionDecoder, a_decoder_name:Optional[str]=None):
	def _get_decoder_sorted_pfs(a_decoder):
		ratemap = a_decoder.pf.ratemap
		CoM_sort_indicies = np.argsort(ratemap.peak_tuning_curve_center_of_masses) # get the indicies to sort the placefields by their center-of-mass (CoM) location
		# CoM_sort_indicies.shape # (n_neurons,)
		return ratemap.pdf_normalized_tuning_curves[CoM_sort_indicies, :]
	a_decoder_name = a_decoder_name or 'new'
	return visualize_heatmap_pyqtgraph(_get_decoder_sorted_pfs(a_decoder), title=f'{a_decoder_name}_pf1Ds', show_value_labels=False, show_xticks=False, show_yticks=False, show_colorbar=False, win=None, defer_show=True)


_display_fn_BasePositionDecoder_visualize_heatmap_pyqtgraph(all_directional_pf1D_Decoder, a_decoder_name='all_directional_pf1D_Decoder')

In [ ]:
# Decode using long_directional_decoder
global_spikes_df, (odd_shuffle_helper, even_shuffle_helper) = RankOrderAnalyses.common_analysis_helper(curr_active_pipeline=curr_active_pipeline, num_shuffles=1000)
spikes_df = deepcopy(global_spikes_df) #.spikes.sliced_by_neuron_id(track_templates.shared_aclus_only_neuron_IDs)
global_replays = TimeColumnAliasesProtocol.renaming_synonym_columns_if_needed(deepcopy(curr_active_pipeline.filtered_sessions[global_epoch_name].replay))
# long_directional_decoding_result: DecodedFilterEpochsResult = long_directional_pf1D_Decoder.decode_specific_epochs(spikes_df, global_replays, decoding_time_bin_size=0.01)
all_directional_decoding_result: DecodedFilterEpochsResult = all_directional_pf1D_Decoder.decode_specific_epochs(spikes_df, global_replays, decoding_time_bin_size=0.01)


In [ ]:
# all_directional_decoding_result.marginal_y_list.p_x_given_n_list
list(all_directional_decoding_result.marginal_y_list[0].keys()) # ['p_x_given_n', 'most_likely_positions_1D']


_out_marginal_y_p_x_given_n = [all_directional_decoding_result.marginal_y_list[i]['p_x_given_n'] for i in np.arange(len(all_directional_decoding_result.marginal_y_list))]
_out_marginal_y_p_x_given_n[0].shape # (4, 26)


_out_marginal_y_p_x_given_n_arr = np.stack(_out_marginal_y_p_x_given_n, axis=-1)


In [ ]:

out = BasicBinnedImageRenderingWindow(a_replay_inst_frs, None, None, name='a_replay_inst_frs', title="Inst Fr for Replay Epoch", variable_label='Inst FR', scrollability_mode=LayoutScrollability.NON_SCROLLABLE)
# out.add_data(row=1, col=0, matrix=binned_x_transition_matrix_higher_order_list[1], xbins=pf1D.xbin_labels, ybins=pf1D.ybin_labels, name='binned_x_transition_matrix^2', title='2nd Order Transition Matrix for binned x (from, to)', variable_label='2nd Order Transition Matrix') # , scrollability_mode=LayoutScrollability.NON_SCROLLABLE
# out.add_data(row=2, col=0, matrix=binned_x_transition_matrix_higher_order_list[2], xbins=pf1D.xbin_labels, ybins=pf1D.ybin_labels, name='binned_x_transition_matrix^3', title='3rd Order Transition Matrix for binned x (from, to)', variable_label='3rd Order Transition Matrix') # , scrollability_mode=LayoutScrollability.NON_SCROLLABLE


In [ ]:
# active_decoding_result = long_directional_decoding_result
active_decoding_result = all_directional_decoding_result

In [ ]:
active_decoding_result.p_x_given_n_list

In [ ]:
# marginalize over the direction to find which direction is most likely
epoch_idx = 0

# long_directional_marginalized
active_decoding_result.marginal_y_list[epoch_idx].p_x_given_n

# np.sum(long_directional_decoding_result.marginal_y_list[epoch_idx].p_x_given_n, axis=0)

time_bins = active_decoding_result.time_bin_containers[epoch_idx]
len(time_bins.centers)
time_bins.num_bins


In [ ]:
active_decoding_result.nbins

In [ ]:

np.sum(active_decoding_result.marginal_y_list[epoch_idx].p_x_given_n, axis=1)/active_decoding_result.time_bin_containers[epoch_idx].num_bins

In [ ]:


 #.shape # (2, n_bins)

# sum across timebins to get total likelihood for each of the two directions
long_relative_direction_likelihoods = np.vstack([(np.sum(active_decoding_result.marginal_y_list[epoch_idx].p_x_given_n, axis=1)/active_decoding_result.time_bin_containers[epoch_idx].num_bins) for epoch_idx in np.arange(active_decoding_result.num_filter_epochs)]) # should get 2 values
display(long_relative_direction_likelihoods.shape) # (n_epochs, 2)
long_relative_direction_likelihoods

# np.all(np.sum(long_relative_direction_likelihoods, axis=1) == 1)
np.sum(long_relative_direction_likelihoods, axis=1) # not sure why some NaN values are getting in there

In [ ]:
is_good_epoch = np.isfinite(np.sum(long_relative_direction_likelihoods, axis=1))
is_good_epoch

In [ ]:
out = BasicBinnedImageRenderingWindow(long_relative_direction_likelihoods, None, None, name='long_relative_direction_likelihoods', title="Directional Decoder Prob for Replay Epochs", variable_label='P_x_given_n', scrollability_mode=LayoutScrollability.NON_SCROLLABLE)

In [ ]:
long_relative_direction_likelihoods

In [ ]:
long_relative_direction_likelihoods

In [ ]:
win = pg.GraphicsWindow()
win.setWindowTitle('DOTS')
# pg.plot(long_relative_direction_likelihoods.T[0], pen=None, symbol='o', symbolPen=None, symbolSize=4, symbolBrush=('b'), name='LR', win=win)
# pg.plot(long_relative_direction_likelihoods.T[1], pen=None, symbol='o', symbolPen=None, symbolSize=4, symbolBrush=('r'), name='RL', win=win)
p1 = win.addPlot() 
curve1 = p1.plot()
curve2 = p1.plot()

#Blue Dots
curve1.setData(long_relative_direction_likelihoods.T[0], pen=None, symbol='o', symbolPen=None, symbolSize=4, symbolBrush=('b'), name='LR')
#Red Dots
curve2.setData(long_relative_direction_likelihoods.T[1], pen=None, symbol='o', symbolPen=None, symbolSize=4, symbolBrush=('r'), name='RL')

In [ ]:
#Blue Dots
curve1.setData(long_relative_direction_likelihoods.T[0], pen=None, symbol='o', symbolPen=None, symbolSize=4, symbolBrush=('b'), name='LR')
#Red Dots
curve2.setData(long_relative_direction_likelihoods.T[1], pen=None, symbol='o', symbolPen=None, symbolSize=4, symbolBrush=('r'), name='RL')

# PhoKamran2023Paper Results

## Figure 1) pf1D Ratemaps, Active set, etc

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot, plot_raster_plot
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap_pyqtgraph # used in `plot_kourosh_activity_style_figure`
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_1_add_replay_epoch_rasters


In [ ]:
# curr_active_pipeline.reload_default_display_functions()

pf1d_compare_graphics, (example_epoch_rasters_L, example_epoch_rasters_S), example_stacked_epoch_graphics, fig_1c_figures_out_dict = PAPER_FIGURE_figure_1_full(curr_active_pipeline) # did not display the pf1

In [ ]:
rdf = jonathan_firing_rate_analysis_result.rdf.rdf
rdf


## Figure 2) `PaperFigureTwo`: LxC/SxC Analyses
Note: this fails when SxC or LxC are empty for this session (as it's not meaningful to produce a comparison bar plot). In this case, aggregate across multiple sessions.

In [ ]:
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PaperFigureTwo

_out_fig_2 = PaperFigureTwo(instantaneous_time_bin_size_seconds=0.01) # 10ms
_out_fig_2.compute(curr_active_pipeline=curr_active_pipeline)
_out_fig_2.display()

## Figure 3) `PAPER_FIGURE_figure_3`: Firing Rate Index and Long/Short Firing Rate Replays v. Laps

In [ ]:
from neuropy.utils.matplotlib_helpers import FormattedFigureText
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _plot_long_short_firing_rate_indicies
# curr_active_pipeline.reload_default_display_functions()

_out, _out2 = PAPER_FIGURE_figure_3(curr_active_pipeline, defer_render=False, save_figure=True)

#  🖼️ Plot a debug view for directional laps. Shows the laps, the pf1Ds, ...


In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.DockAreaWrapper import DockAreaWrapper, PhoDockAreaContainingWindow
from pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig import DisplayColorsEnum, LongShortDisplayConfigManager
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.GraphicsWidgets.EpochsEditorItem import EpochsEditor # perform_plot_laps_diagnoser
from pyphoplacecellanalysis.External.pyqtgraph.dockarea.Dock import Dock, DockDisplayConfig
from pyphoplacecellanalysis.GUI.PyQtPlot.DockingWidgets.DynamicDockDisplayAreaContent import CustomDockDisplayConfig

epochs_editor = EpochsEditor.init_from_session(global_session, include_velocity=False, include_accel=False)
root_dockAreaWindow, app = DockAreaWrapper.wrap_with_dockAreaWindow(epochs_editor.plots.win, None, title='Pho Directional Laps Templates')

# track_templates.long_LR_decoder.pf

def _get_decoder_sorted_pfs(a_decoder):
	ratemap = a_decoder.pf.ratemap
	CoM_sort_indicies = np.argsort(ratemap.peak_tuning_curve_center_of_masses) # get the indicies to sort the placefields by their center-of-mass (CoM) location
	# CoM_sort_indicies.shape # (n_neurons,)
	return ratemap.pdf_normalized_tuning_curves[CoM_sort_indicies, :]

decoders_dict = {'long_LR': track_templates.long_LR_decoder,
	'long_RL': track_templates.long_RL_decoder,
	'short_LR': track_templates.short_LR_decoder,
	'short_RL': track_templates.short_RL_decoder,
}

## Plot the placefield 1Ds as heatmaps and then wrap them in docks and add them to the window:
_out_pf1D_heatmaps = {}
for a_decoder_name, a_decoder in decoders_dict.items():
	_out_pf1D_heatmaps[a_decoder_name] = visualize_heatmap_pyqtgraph(_get_decoder_sorted_pfs(a_decoder), title=f'{a_decoder_name}_pf1Ds', show_value_labels=False, show_xticks=False, show_yticks=False, show_colorbar=False, win=None, defer_show=True)

even_dock_config = CustomDockDisplayConfig(custom_get_colors_callback_fn=DisplayColorsEnum.Laps.get_even_dock_colors)
odd_dock_config = CustomDockDisplayConfig(custom_get_colors_callback_fn=DisplayColorsEnum.Laps.get_odd_dock_colors)


_out_dock_widgets = {}
dock_configs = (even_dock_config, odd_dock_config, even_dock_config, odd_dock_config)
dock_add_locations = (['left'], ['left'], ['right'], ['right'])

for i, (a_decoder_name, a_heatmap) in enumerate(_out_pf1D_heatmaps.items()):
	_out_dock_widgets[a_decoder_name] = root_dockAreaWindow.add_display_dock(identifier=a_decoder_name, widget=a_heatmap[0], dockSize=(300,200), dockAddLocationOpts=dock_add_locations[i], display_config=dock_configs[i])


# Outputs: root_dockAreaWindow, app, epochs_editor, _out_pf1D_heatmaps, _out_dock_widgets

# 🟢🖼️ 2023-10-27 - Example validation of directional shuffles with multi-raster plot 

In [ ]:
from pyphoplacecellanalysis.General.Mixins.DataSeriesColorHelpers import DataSeriesColorHelpers
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.GraphicsWidgets.DirectionalTemplatesRastersDebugger import _debug_plot_directional_template_rasters
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.GraphicsWidgets.DirectionalTemplatesRastersDebugger import build_selected_spikes_df
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.GraphicsWidgets.DirectionalTemplatesRastersDebugger import add_selected_spikes_df_points_to_scatter_plot
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.DockAreaWrapper import DockAreaWrapper

from pyphocorehelpers.print_helpers import generate_html_string # used for `plot_long_short_surprise_difference_plot`

In [ ]:
## Inputs: curr_active_pipeline, track_templates, global_replays, owning_pipeline_reference
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
# global_spikes_df = deepcopy(curr_active_pipeline.computation_results[global_epoch_name]['computed_data'].pf1D.spikes_df)
global_spikes_df = deepcopy(curr_active_pipeline.filtered_sessions[global_epoch_name].spikes_df)
global_laps = deepcopy(curr_active_pipeline.filtered_sessions[global_epoch_name].laps) # .trimmed_to_non_overlapping()
global_laps_epochs_df = global_laps.to_dataframe()
active_epochs_df = global_laps_epochs_df.copy()

odd_display_outputs, even_display_outputs = _debug_plot_directional_template_rasters(global_spikes_df, active_epochs_df, track_templates)
odd_app, odd_win, odd_plots, odd_plots_data, odd_on_update_active_epoch, odd_on_update_active_scatterplot_kwargs = odd_display_outputs
even_app, even_win, even_plots, even_plots_data, even_on_update_active_epoch, even_on_update_active_scatterplot_kwargs = even_display_outputs
# Build the wrapping window using `DockAreaWrapper`:
active_root_main_widget = odd_win.window()
root_dockAreaWindow, app = DockAreaWrapper.wrap_with_dockAreaWindow(active_root_main_widget, even_win, title='Pho Rank-Order Epochs Debugger')


def on_update_active_epoch(an_epoch_idx, an_epoch):
    """ captures: odd_on_update_active_epoch, even_on_update_active_epoch, root_dockAreaWindow """
    odd_on_update_active_epoch(an_epoch_idx, an_epoch=an_epoch)
    even_on_update_active_epoch(an_epoch_idx, an_epoch=an_epoch)
    root_dockAreaWindow.setWindowTitle(f'Pho Rank-Order Epochs Debugger: Epoch[{an_epoch_idx}]')

def on_update_epoch_IDX(an_epoch_idx):
    """ captures on_update_active_epoch, active_epochs_df to extract the epoch time range and call `on_update_active_epoch` """
    # curr_epoch_spikes = spikes_df[(spikes_df.new_lap_IDX == an_epoch_idx)]
    curr_epoch_df = active_epochs_df[(active_epochs_df.lap_id == (an_epoch_idx+1))]
    curr_epoch = list(curr_epoch_df.itertuples())[0]

    on_update_active_epoch(an_epoch_idx, curr_epoch)

## Build the selected spikes df:

## Laps: RL_laps_epoch_selected_spikes_fragile_linear_neuron_IDX_dict, LR_laps_epoch_selected_spikes_fragile_linear_neuron_IDX_dict
## Ripples: RL_ripple_evts_epoch_selected_spikes_fragile_linear_neuron_IDX_dict, LR_ripple_evts_epoch_selected_spikes_fragile_linear_neuron_IDX_dict

(even_selected_spike_df, even_neuron_id_to_new_IDX_map), (odd_selected_spike_df, odd_neuron_id_to_new_IDX_map) = build_selected_spikes_df(track_templates, active_epochs_df, RL_laps_epoch_selected_spikes_fragile_linear_neuron_IDX_dict, LR_laps_epoch_selected_spikes_fragile_linear_neuron_IDX_dict)

## Add the spikes
add_selected_spikes_df_points_to_scatter_plot(plots_data=odd_plots_data, plots=odd_plots, selected_spikes_df=deepcopy(odd_selected_spike_df), _active_plot_identifier = 'long_odd')
add_selected_spikes_df_points_to_scatter_plot(plots_data=odd_plots_data, plots=odd_plots, selected_spikes_df=deepcopy(odd_selected_spike_df), _active_plot_identifier = 'short_odd')
add_selected_spikes_df_points_to_scatter_plot(plots_data=even_plots_data, plots=even_plots, selected_spikes_df=deepcopy(even_selected_spike_df), _active_plot_identifier = 'long_even')
add_selected_spikes_df_points_to_scatter_plot(plots_data=even_plots_data, plots=even_plots, selected_spikes_df=deepcopy(even_selected_spike_df), _active_plot_identifier = 'short_even')


In [ ]:
odd_display_outputs

print(locals())

In [ ]:
 

all_plots = odd_plots + even_plots # TypeError: unsupported operand type(s) for +: 'RenderPlots' and 'RenderPlots'

### Ripples Debugger:

In [ ]:
## Inputs: curr_active_pipeline, track_templates, global_replays, owning_pipeline_reference

def _get_shared_data(epoch_idx: int):
    # i_str = generate_html_string('i', color='white', bold=True)
    # j_str = generate_html_string('j', color='red', bold=True)
    # title_str = generate_html_string(f'JSD(p_x_given_n, pf[{i_str}]) - JSD(p_x_given_n, pf[{j_str}]) where {j_str} non-firing')
	return [generate_html_string(str(round(arr[epoch_idx], ndigits=3)), color='white', bold=True) for arr in (LR_ripple_evts_long_z_score_values, RL_ripple_evts_long_z_score_values, LR_ripple_evts_short_z_score_values, RL_ripple_evts_short_z_score_values)]


global_spikes_df = deepcopy(curr_active_pipeline.filtered_sessions[global_epoch_name].spikes_df)

global_ripples_epochs_df = global_replays.to_dataframe()
active_epochs_df = global_ripples_epochs_df.copy()

odd_display_outputs, even_display_outputs = _debug_plot_directional_template_rasters(global_spikes_df, active_epochs_df, track_templates)
odd_app, odd_win, odd_plots, odd_plots_data, odd_on_update_active_epoch, odd_on_update_active_scatterplot_kwargs = odd_display_outputs
even_app, even_win, even_plots, even_plots_data, even_on_update_active_epoch, even_on_update_active_scatterplot_kwargs = even_display_outputs

# Build the wrapping window using `DockAreaWrapper`:
active_root_main_widget = odd_win.window()
root_dockAreaWindow, app = DockAreaWrapper.wrap_with_dockAreaWindow(active_root_main_widget, even_win, title='Pho Rank-Order Epochs Debugger')

def on_update_active_epoch(an_epoch_idx, an_epoch):
    """ captures: odd_on_update_active_epoch, even_on_update_active_epoch, root_dockAreaWindow """
    odd_on_update_active_epoch(an_epoch_idx, an_epoch=an_epoch)
    even_on_update_active_epoch(an_epoch_idx, an_epoch=an_epoch)
    root_dockAreaWindow.setWindowTitle(f'Pho Rank-Order Epochs Debugger: Epoch[{an_epoch_idx}]')


def on_update_epoch_IDX(an_epoch_idx):
    """ captures on_update_active_epoch, active_epochs_df to extract the epoch time range and call `on_update_active_epoch`, _get_shared_data, odd_plots, even_plots """
    # curr_epoch_spikes = spikes_df[(spikes_df.new_lap_IDX == an_epoch_idx)]
    # curr_epoch_df = active_epochs_df[(active_epochs_df.lap_id == (an_epoch_idx+1))]
    # curr_epoch_df = active_epochs_df[(active_epochs_df.label.astype(float) == float(an_epoch_idx))]
    # curr_epoch_df = active_epochs_df[(active_epochs_df.index.astype(float) == float(an_epoch_idx))]
    curr_epoch_df = active_epochs_df[active_epochs_df.index == active_epochs_df.index.to_numpy()[an_epoch_idx]]
    curr_epoch = list(curr_epoch_df.itertuples())[0] # Pandas(Index=233, start=799.9181619619485, stop=800.1395608885214, label='235', duration=0.2213989265728742)
    
    ## Add values as labels:
    curr_epoch_raw_values = _get_shared_data(an_epoch_idx)
    long_odd_val, long_even_val, short_odd_val, short_even_val = curr_epoch_raw_values
    odd_plots['ax']['long_odd'].getAxis('left').setLabel(f'long_odd: {long_odd_val}')
    odd_plots['ax']['short_odd'].getAxis('left').setLabel(f'short_odd: {short_odd_val}')
    even_plots['ax']['long_even'].getAxis('left').setLabel(f'long_even: {long_even_val}')
    even_plots['ax']['short_even'].getAxis('left').setLabel(f'short_even: {short_even_val}')
    
    # for ax_name in ('long_odd', 'short_odd'):
    # 	odd_plots['ax'][ax_name].getAxis('left').setLabel(f'{ax_name}: {}')

    on_update_active_epoch(an_epoch_idx, curr_epoch)

## Build the selected spikes df:

## Laps: RL_laps_epoch_selected_spikes_fragile_linear_neuron_IDX_dict, LR_laps_epoch_selected_spikes_fragile_linear_neuron_IDX_dict
## Ripples: RL_ripple_evts_epoch_selected_spikes_fragile_linear_neuron_IDX_dict, LR_ripple_evts_epoch_selected_spikes_fragile_linear_neuron_IDX_dict

(even_selected_spike_df, even_neuron_id_to_new_IDX_map), (odd_selected_spike_df, odd_neuron_id_to_new_IDX_map) = build_selected_spikes_df(track_templates, active_epochs_df, RL_ripple_evts_epoch_selected_spikes_fragile_linear_neuron_IDX_dict, LR_ripple_evts_epoch_selected_spikes_fragile_linear_neuron_IDX_dict)

## Add the spikes
add_selected_spikes_df_points_to_scatter_plot(plots_data=odd_plots_data, plots=odd_plots, selected_spikes_df=deepcopy(odd_selected_spike_df), _active_plot_identifier = 'long_odd')
add_selected_spikes_df_points_to_scatter_plot(plots_data=odd_plots_data, plots=odd_plots, selected_spikes_df=deepcopy(odd_selected_spike_df), _active_plot_identifier = 'short_odd')
add_selected_spikes_df_points_to_scatter_plot(plots_data=even_plots_data, plots=even_plots, selected_spikes_df=deepcopy(even_selected_spike_df), _active_plot_identifier = 'long_even')
add_selected_spikes_df_points_to_scatter_plot(plots_data=even_plots_data, plots=even_plots, selected_spikes_df=deepcopy(even_selected_spike_df), _active_plot_identifier = 'short_even')


In [ ]:
epoch_idx = 299
on_update_epoch_IDX(epoch_idx)
# programmatically_select_epoch(epoch_idx)